

Credit modelling using linancial lending data from Lending Club
------------------------------------------

Modeling Credit status by making predictions about whether or not a loan will be paid off on time, which is contained in the *loan_status* column of the clean dataset.

40000 approved loan data with loan status feature from 2017 to 2011 are analyzed in the project. The datasets using in the project is in the Data folder.

Using False Positive Rate(fpr) and Accuracy as error metric. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid.

Prediction Goal: Minimize the fpr results.


## Reading Raw Data

* removing the first line:
    * because it contains the extraneous text Notes offered by Prospectus 
    (https://www.lendingclub.com/info/prospectus.action) instead of the column titles, which prevents the dataset from being parsed by the pandas library properly
* removing the desc column:
    * which contains a long text explanation for each loan
* removing the url column:
    * which contains a link to each loan on Lending Club which can only be accessed with an investor account
* removing all columns containing more than 50% missing values:
    * which allows us to move faster since we can spend less time trying to fill these values

In [1]:
import pandas as pd
import numpy as np


#######Filtering Function########
def Filter(df, key, value):
	return df[df[key] == value]
pd.DataFrame.Filter = Filter


loans_2007 = pd.read_csv('Data/rawData.csv', skiprows=1)
half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)
loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)
loans_2007.to_csv('Data/loans_2007.csv', index=False)


/home/dawei/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Cleaning Data

Breaking up the columns into 3 groups of 18 columns and use the data dictionary to become familiar with what each column represents. After understanding each feature, we want to pay attention to any features that:

* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information

### Cleaning redundent features and features leak future information

In [2]:
loan_2007 = pd.read_csv('Data/loans_2007.csv')

## Remove redundent and leak feature in first group
loans_2007 = loans_2007.drop(["pymnt_plan","id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

## Remove redundent and leak featured in second group
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

## Remove redundent and leak featured in third group
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)


/home/dawei/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Cleaning target Column

* Discovering the imbalance of Loan_status column
* Converting the loan status string to binary classfication features

In [4]:
loans_2007.loan_status.value_counts()


Fully Paid                                             33314
Charged Off                                             5640
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  781
Does not meet the credit policy. Status:Charged Off      761
In Grace Period                                           22
Late (31-120 days)                                        17
Late (16-30 days)                                          8
Default                                                    4
Name: loan_status, dtype: int64

In [5]:
## Convert the Loan Status string to numeric value

loans_2007=loans_2007[(loans_2007.loan_status == 'Fully Paid') | (loans_2007.loan_status=='Charged Off')]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

### Remove Single Value Columns

In [6]:
## Remove Single Value Columns

orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)


['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


### Handing Missing Value

In [7]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

## Drop pub_rec_bankruptcies Column since it has 697 Null value
loans_2007 = loans_2007.drop("pub_rec_bankruptcies", axis=1)
loans_2007 = loans_2007.dropna(axis=0)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


### Converting the Object Columns to Numerical 

* Drop % Sign
* Convert categorical features to Numeric features
* Convert One-hot features

In [8]:
## Printing Object Columns

object_columns_df=loans_2007.select_dtypes(include=['object'])
loans_2007 = loans_2007.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)


In [9]:
## Delete % Sign
loans_2007["int_rate"] = loans_2007["int_rate"].str.rstrip("%").astype("float")
loans_2007["revol_util"] = loans_2007["revol_util"].str.rstrip("%").astype("float")


## Convert categorical features to Numeric features
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}


loans_2007 = loans_2007.replace(mapping_dict)

In [10]:
## Convert One-hot Features

col=['home_ownership','verification_status','purpose','term']
for c in col:
    loans_2007[c]=loans_2007[c].astype('category')
dummy_df=pd.get_dummies(loans_2007[col])
loans_2007=pd.concat([loans_2007,dummy_df],axis=1)
dummy_df.head()
loans_2007 = loans_2007.drop(col, axis=1)

## Exporting the processed Data

In [12]:
loans_2007.to_csv('Processed/Cleaned_loans_2007.csv', index=False)

## Next thing to do:

We can tweak the penalties further.

We can try models other than a random forest and logistic regression.

We can use some of the columns we discarded to generate better features.

We can ensemble multiple models to get more accurate predictions.

We can tune the parameters of the algorithm to achieve higher performance.